# Entity Explorer - Account
 <details>
     <summary>&nbsp;<u>Details...</u></summary>

 **Notebook Version:** 2.0<br>
 **Python Version:** Python 3.8+<br>
 **Required Packages**: kqlmagic, msticpy, pandas, numpy, matplotlib, networkx, ipywidgets, ipython, dnspython, ipwhois, folium, maxminddb_geolite2, msticnb<br>

 **Data Sources Required**:
 - Log Analytics - SecurityAlert, SecurityEvent, HuntingBookmark, Syslog, AAD SigninLogs, AzureActivity, OfficeActivity, ThreatIndicator
 - (Optional) - VirusTotal, AlienVault OTX, IBM XForce, Open Page Rank, (all require accounts and API keys)
 </details>

 Brings together a series of queries and visualizations to help you determine the security state of an Account. The account can be a Windows or Linux account or an Azure Active Directory/Office 365 account.

The Notebook contains sections for reviewing activity for Host accounts (Linux and Windows) and for Azure Active Directory accounts. It also has a general section that looks for related items independent of the account type.

<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
    <h1>Contents<span class="tocSkip"></span></h1>
    <div class="toc">
      <ul class="toc-item">
        <li><span><a href="#Hunting-Hypothesis">Hunting Hypothesis</a></span></li>
        <li><span><a href="#Notebook-initialization">Notebook initialization</a></span></li>
        <li><span><a href="#Enter-account-name-and-query-time">Enter account
              name and query time</a></span></li>
        <li><span><a href="#Run-the-main-account_summary-function">Run the main
              account_summary function</a></span></li>
        <li><span><a href="#Retrieve-additional-data-for-the-selected-account">Retrieve
              additional data for the selected account</a></span></li>
        <li><span><a href="#Browse-alerts">Browse alerts</a></span></li>
        <li><span><a href="#Browse-other-event-data">Browse other event data</a></span></li>
        <li><span><a href="#More-operations-available">
              Using additional properties and methods of the result object<br>
            </a></span></li>
        <li><span><a href="#More-information">More information</a></span></li>
      </ul>
    </div>
    
  </body>
</html>


## Hunting Hypothesis
Our broad initial hunting hypothesis is that a we have received account name entity which is suspected to be compromised and is being used malicious manner in internal networks, we will need to hunt from a range of different positions to validate or disprove this hypothesis.

---
# Notebook initialization
This should complete without errors. If you encounter errors or warnings look at the following notebooks:

- <a href="https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb">Getting Started Notebook</a>
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

<details>
    <summary>&nbsp;<u>Details...</u></summary>
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

If you are running in the Azure Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb)
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)
</details>

In [2]:
from pathlib import Path
from IPython.display import display, HTML
from datetime import datetime, timedelta, timezone

REQ_PYTHON_VER = "3.8"
REQ_MSTICPY_VER = "1.7.5"


# %pip install --upgrade msticpy
display(HTML("<h3>Starting Notebook setup...</h3>"))

import msticpy
msticpy.init_notebook(
    namespace=globals(),
    additional_packages=["msticnb>=0.2"],
    verbosity=0,
);


All packages are already installed


In [3]:
# papermill default parameters
ws_name = "Default"
account_name = ""
account_types = ["All"]  # Windows, Linux, Azure, All
end = datetime.now(timezone.utc)
start = end - timedelta(days=2)


### Get Workspace and Authenticate
<details>
    <summary>&nbsp;<u>Details...</u></summary>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.<br>
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. <br>
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. <br>
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.<br>


To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [4]:
print("Configured workspaces: ", ", ".join(msticpy.settings.get_config("AzureSentinel.Workspaces").keys()))
ws_param = nbwidgets.GetText(
    description="Workspace Name",
    default=ws_name,
)
ws_param

Configured workspaces:  ASIHuntOMSWorkspaceV4, CCIS, Centrica, CyberSecuritySoc, Default, GovCyberSecuritySOC, NationalGrid, RedmondSentinelDemoEnvironment


Text(value='Default', description='Workspace Name', layout=Layout(width='50%'), style=DescriptionStyle(descripâ€¦

In [5]:
from msticpy.common.timespan import TimeSpan

# Authentication
qry_prov = QueryProvider(data_environment="MSSentinel")
qry_prov.connect(WorkspaceConfig(workspace=ws_param.value))

nb_timespan = TimeSpan(start, end)
qry_prov.query_time.timespan = nb_timespan
md("<hr>")
md("Confirm time range to search", "bold")
qry_prov.query_time

Please wait. Loading Kqlmagic extension...done
Connecting... 

connected


#### Authentication and Configuration problems?

If you are having problems, expand the details section below
<br>
<details>
    <summary>Click for details about configuring your authentication parameters</summary>
    
    
The notebook is expecting your Azure Sentinel Tenant ID and Workspace ID to be configured in one of the following places:
- `config.json` in the current folder
- `msticpyconfig.yaml` in the current folder or location specified by `MSTICPYCONFIG` environment variable.
    
For help with setting up your configuration (if this hasn't been done automatically) see the [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb) notebook in the root folder of your Azure-Sentinel-Notebooks project.
</details>

## Import and initialize notebooklets

These are needed for many of the notebook operations

In [6]:
import msticnb as nb

nb.init(query_provider=qry_prov)
pivot.timespan = qry_prov.query_time.timespan

Notebooklets: 9 notebooklets loaded.
Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Notebooklets: Loaded providers: AzureSentinel, geolitelookup, tilookup


# Enter account name and query time window
Type the account name that you want to search for and the time bounds over which you want to search. 

You can specify the account as:

- a simple user name (e.g. `alice`)
- a user principal name (`alice@contoso.com`)
- a qualified windows user name `mydomain\alice`

In the second two cases the domain qualifier will be stripped off before the search. The search is not case sensitive and will match full substrings. E.g. `bob` will match `domain\bob` and `bob@contoso.com` but not `bobg` or `bo`.

In [7]:
account_txt = nbwidgets.GetText(prompt='Enter the Account name to search for:', value=account_name)

display(account_txt)
md("<hr>")

Text(value='', description='Enter the Account name to search for:', layout=Layout(width='50%'), style=Descriptâ€¦

In [8]:
from msticnb.nb.azsent.account.account_summary import AccountType
acct_types = [item.name for item in AccountType]
import ipywidgets as widgets
acct_types_select = widgets.SelectMultiple(
    description="Select Account types to search",
    options=acct_types,
    value=account_types or acct_types,
    **WIDGET_DEFAULTS,
)
acct_types_select


SelectMultiple(description='Select Account types to search', index=(6,), layout=Layout(width='95%'), options=(â€¦

# Run the main *account_summary* function

If more than one matching account name is found, all matches will be shown. 

You can select each one to view more details about the account.

> Note: this is run as a pivot function from the Account entity.
> You can also run it directly as a Notebooklet function by 
> creating and instance of `nb.nblt.azsent.account.AccountSummary()`
> and executing the `run()` method

```python
>>> acct_nb = nb.nblt.azsent.account.AccountSummary()
>>> acct_result = acct_nb.run(pivot.timespan, "account_name")  # subtitute the "account_name" with the name to search
```

In [9]:
Account = entities.Account
acc_result = Account.nblt.account_summary(
    value=account_txt.value,
    account_types=acct_types_select.value
)

Loading BokehJS ...

Account(Name=kelsea@seccxp.ninja, UPNSuffix=seccxp.ninja, AadTenantId=4b2462a4-bbee-495a...)

,TimeGenerated,TenantId,SourceSystem,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationContextClassReferences,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,...,ResourceIdentity,ResourceServicePrincipalId,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,UniqueTokenIdentifier,SessionLifetimePolicies,AutonomousSystemNumber,Type,AccountName,Source
0,2022-04-13 16:25:12.987000+00:00,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,be8f88b0-e9f1-47e5-9374-4309bff23d1c,Microsoft.aadiam,Microsoft.aadiam,,Kenyetta Elsea,4,AU,,CAttack,0d465159-840f-4192-a055-3e9141695cb9,"[{""id"":""urn:microsoft:req1"",""detail"":""previouslySatisfied""}]",[],,"[{""key"":""Azure AD App Authentication Library"",""value"":""Family: MSAL Library: MSAL.NET 4.14.0.0 P...",...,00000003-0000-0000-c000-000000000000,f9d938f5-d874-41cd-b8bc-482617434b3d,,,{'errorCode': 0},,AzureAD,,Kenyetta Elsea,f344d9bb-2a11-4ac9-a270-55b342399fcb,kelsea@seccxp.ninja,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,N2YzMjNkZmMtYjhiOS00MTI3LWE1ZjgtNmJiNjJiMzRjNTAw,,8075,SigninLogs,kelsea@seccxp.ninja,AzureActiveDirectory


In [10]:
if len(acc_result.account_selector.options) > 1:
    md_warn("Multiple matches found for account. Running on first listed account")

# Retrieve additional data for the selected account

The result returned from the last cell has a number of properties and methods that you can use
to retrieve and view further information.

The main one for this notebooklet is `get_additional_data`.

In [11]:
acc_result.get_additional_data()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

OperationCount  \
UserPrincipalName   Type           IPAddress                  AppResourceProvider        UserType                   
kelsea@seccxp.ninja OfficeActivity 20.190.142.169             Exchange                   Regular               32   
                                   20.190.142.170             Exchange                   Regular               42   
                                   20.190.142.172             Exchange                   Regular               36   
                                   20.37.213.236:63603        Exchange                   Admin                  1   
                                   2603:10b6:208:350::7       Exchange                   Regular                4   
                                   2603:10b6:208:357::21      Exchange                   Regular                1   
                                   2603:10b6:408:10d:cafe::a  Exchange                   Regular                1   
                                   2603:10b6:408:131:cafe::74 Exchange                   Regular                1   
                                   2603:10b6:408:148::20      Exchange                   Regular                1   
                                   2603:10b6:408:16a::13      Exchange                   Regular               70   
                                   52.96.203.21               Exchange                   Regular                1   
                                   52.96.203.93               Exchange                   Regular                1   
                    SigninLogs     20.37.212.69               CAttack                    Member                35   
                                   20.37.213.236              CAttack                    Member                49   
                                                              Office 365 Exchange Online Member                 1   
                                   20.37.253.233              CAttack                    Member                58   
                                   20.37.255.76               CAttack                    Member                36   
                                   20.53.9.84                 CAttack                    Member                51   
                                   20.70.208.63               CAttack                    Member                47   

                                                                                                              OperationTypes  \
UserPrincipalName   Type           IPAddress                  AppResourceProvider        UserType                              
kelsea@seccxp.ninja OfficeActivity 20.190.142.169             Exchange                   Regular   [Send, MailItemsAccessed]   
                                   20.190.142.170             Exchange                   Regular   [Send, MailItemsAccessed]   
                                   20.190.142.172             Exchange                   Regular   [Send, MailItemsAccessed]   
                                   20.37.213.236:63603        Exchange                   Admin            [Remove-InboxRule]   
                                   2603:10b6:208:350::7       Exchange                   Regular         [MailItemsAccessed]   
                                   2603:10b6:208:357::21      Exchange                   Regular         [MailItemsAccessed]   
                                   2603:10b6:408:10d:cafe::a  Exchange                   Regular                    [Create]   
                                   2603:10b6:408:131:cafe::74 Exchange                   Regular                    [Create]   
                                   2603:10b6:408:148::20      Exchange                   Regular         [MailItemsAccessed]   
                                   2603:10b6:408:16a::13      Exchange                   Regular         [MailItemsAccessed]   
                                   52.96.203.21               Exchange                   Regular         

Count  \
Source               Operation         UserAgent IpAddress                  AsnDescription                  CountryCode          
AzureActiveDirectory Sign-in activity            20.37.212.69               MICROSOFT-CORP-MSN-AS-BLOCK, US AU              35   
                                                 20.37.213.236              MICROSOFT-CORP-MSN-AS-BLOCK, US AU             196   
                                                 20.37.253.233              MICROSOFT-CORP-MSN-AS-BLOCK, US AU              58   
                                                 20.37.255.76               MICROSOFT-CORP-MSN-AS-BLOCK, US AU              36   
                                                 20.53.9.84                 MICROSOFT-CORP-MSN-AS-BLOCK, US AU              51   
                                                 20.70.208.63               MICROSOFT-CORP-MSN-AS-BLOCK, US AU              47   
                                       BAV2ROPC  20.37.213.236              MICROSOFT-CORP-MSN-AS-BLOCK, US AU               4   
Office365            Create                      2603:10b6:408:10d:cafe::a  NA                              US               1   
                                                 2603:10b6:408:131:cafe::74 NA                              US               1   
                     MailItemsAccessed           20.190.142.169             MICROSOFT-CORP-MSN-AS-BLOCK, US AU               9   
                                                 20.190.142.170             MICROSOFT-CORP-MSN-AS-BLOCK, US AU               6   
                                                 20.190.142.172             MICROSOFT-CORP-MSN-AS-BLOCK, US AU              11   
                                                 2603:10b6:208:350::7       NA                              US               4   
                                                 2603:10b6:208:357::21      NA                              US               1   
                                                 2603:10b6:408:148::20      NA                              US               1   
                                                 2603:10b6:408:16a::13      NA                              US              70   
                                                 52.96.203.21               MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   
                                                 52.96.203.93               MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   
                     Send                        20.190.142.169             MICROSOFT-CORP-MSN-AS-BLOCK, US AU              23   
                                                 20.190.142.170             MICROSOFT-CORP-MSN-AS-BLOCK, US AU              36   
                                                 20.190.142.172             MICROSOFT-CORP-MSN-AS-BLOCK, US AU              25   

                                                                                                                                          FirstOperation  \
Source               Operation         UserAgent IpAddress                  AsnDescription                  CountryCode                                    
AzureActiveDirectory Sign-in activity            20.37.212.69               MICROSOFT-CORP-MSN-AS-BLOCK, US AU          2022-04-11 16:55:16.334000+00:00   
                                                 20.37.213.236              MICROSOFT-CORP-MSN-AS-BLOCK, US AU          2022-04-11 18:08:23.163000+00:00   
                                                 20.37.253.233              MICROSOFT-CORP-MSN-AS-BLOCK, US AU          2022-04-11 17:03:13.837000+00:00   
                                                 20.37.255.76               MICROSOFT-CORP-MSN-AS-BLOCK, US AU          2022-04-11 19:31:22.616000+00:00   
                                                 20.53.9.84                 MICROSOFT-CORP-MSN-AS-BLOCK, US AU          2022-04-11 20:42:28.973000+00:00   
                                      

# Browse alerts
If there are any alerts referencing this account name they can be viewed
by calling the `acc_result.browse_alerts()` function.

In [12]:
acc_result.notebooklet.browse_alerts()


,0
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
TimeGenerated,2022-04-12 00:50:05.465000+00:00
AlertDisplayName,Impossible Travel Activity by User
AlertName,Impossible Travel Activity by User
Severity,Medium
Description,Test
ProviderName,ASI Scheduled Alerts
VendorName,Microsoft
VendorOriginalId,2bcbc749-d89b-4a0c-a252-3dd48ba40bdc
SystemAlertId,c19a7585-0c6a-3cfa-c374-9abe9590fa70


# Browse other event data

You can use a simple view to make it easier to see the details of individual events
by calling the "view_events" method.

You need to supply the name of the result attribute that you want to view plus
one or more summary columns (as a list of strings).

In [13]:
print(
    "Available data properties\n",
    acc_result.data_properties()
)

Available data properties
 ['account_activity', 'related_alerts', 'azure_activity', 'azure_activity_summary', 'ip_summary', 'ip_all_data']


In [17]:
acc_result.view_events(
    attrib="azure_activity",  # the result attribute to view
    # summary_cols controls which cols are use to create the summary
    # for the select list
    summary_cols=["SourceSystem", "Operation", "IPAddress"]
)

,291
TenantId,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,OfficeActivityManager
TimeGenerated,2022-04-11 16:55:28+00:00
Operation,MailItemsAccessed
AppResourceProvider,Exchange
AppId,13937bba-652e-4c46-b222-3003f4d1ff97
IPAddress,2603:10b6:408:16a::13
UserId,Kelsea@seccxp.ninja
UserPrincipalName,kelsea@seccxp.ninja
UserType,Regular


# More operations available

---

## Browse events with alternative sorting
You can pass a DatFrame to view events instead of an attribute name.

This means that you can apply sorting or filtering of the data before viewing it.
Here is an example sorting by IPAddress.

```python
acc_result.view_events(
    data=acc_result.azure_activity.sort_values("IPAddress"),
    summary_cols=["SourceSystem", "Operation", "IPAddress"]
)
```

---

## Additional properties and methods of the result object

These are static properties - usually DataFrames or visualizations. 
You can access each of these to see or manipulate the retrieved data.

To see help on the available attributes type:
```python
>>> help(acc_result)
```
To see the available methods type:
```python
>>> acc_result.list_methods()
```
> Note, for the AccountSummary notebooklet, the two main data retrieval methods are:<br>
> - run<br>
> - get_additional_data<br>
> There are several other methods that allow you to view individual plots
> or subsets of the data (such as alerts).

To view help on a specific method type:
```python
>>> help(acc_result.method_name)
```

To run a method
```python
acc_result.display_alert_timeline()
```

---

## Viewing the Result class
You can view all of the data in the results class by "running" it in a cell

> Note: This produces a lot of output.<br>
> Due to the way Jupyter display Javascript objects the plots may
> appear out of order.

```ipython
acc_result
```


---

## Using Pivots to get more context information

You can run a pivot function on the account summary results
to get additional context on the data.

Here is an example of looking up Whois information for Azure IPAddress requests.

```python
whois_df = (
    acc_result                      # the results object
    .azure_activity[["IPAddress"]]  # the property and the column we want
    .drop_duplicates()              # drop duplicates
    .mp_pivot.run(                  # run the pivot function IpAddress 'whois' function
        IpAddress.util.whois, column="IPAddress" 
    )
)
whois_df
```

---

## Use other notebooklets and pivots functions to drill down on other entities

You may want to drill down on other entities in the Account data.
You can use methods of the IpAddress or Host entities, for example,
to look at these in more detail.

Run the ip_address_summary notebooklet pivot
```python
IpAddress = entities.IpAddress
ip_result = IpAddress.nblt.ip_address_summary("157.56.162.53")
```

View the TI results
```python
ip_result.browse_ti_results()
```

---

# More information:

## Notebooklets and Pivots
[Notebooklets](https://msticnb.readthedocs.io/en/latest/)

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

## Notebook/MSTICPy configuration
[Getting Started]((https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb)
[MSTICPy Configuration guide](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)

[ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)